Imports...

In [9]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import os
import unicodedata
import sys
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import nltk.data
from gensim.models import Word2Vec

In [10]:
import nltk.data
# nltk.download()

Vectorize Function

In [3]:
def vectorize(doc, vocabulary):
    # It just splits the string on a space and then counts the number of times that terms happen. 
    bag_of_words = Counter(doc.split(' '))
    # The counter object will be translated into a vector.
    # And the next line creates a numpy vector with space for those entries. 
    doc_vector = np.zeros(len(vocabulary))
    # The next for loop is very interesting. Although the doc may have words that are not in 
    # the vocabulary, it is only for words in the vocabulary that the vector is made. 
    # word_index is an integer and indexes the doc_vector. 
    for word_index, word in enumerate(vocabulary):
        if word in bag_of_words:
            doc_vector[word_index] += bag_of_words[word]
    return doc_vector

Other functions (combine?)


In [11]:
class Book:
    
    def __init__(self, book):
        self.title = ''
        self.author = ''
        #             self.title = book[8].strip().replace('Title: ','')
        #             self.author = book[1].strip()
        #             self.region = book[4].strip()
        #             self.lifespan = book[2].strip()
        #             self.topics = [topic.strip() for topic in book[5].split(',')]
        #             self.type = book[7]
        #             self.text = "".join(book[11:])
        #             self.clean_text = ""
        #             self.author = book[10 .strip().replace('Author: ','')
        n = len(book)
        lim = 30
        if lim > n:
            lim = n
        # Project Gutenburg has no way to determine front matter,
        # so cut first 5% to avoid it
        start = int(n - 0.95*n)
        for line in book[:lim]:
            if 'Title: ' in line:
                self.title = line.strip().replace('Title: ','')
            if 'Author: ' in line:
                self.author = line.strip().replace('Author: ','')
        # start set to strip off various lengths of front-matter
        temp_text = "".join(book[start:]).decode('utf-8')
        # -19350 to strip off the Project Gutenberg donation requests
        self.text = temp_text[0:len(temp_text) - 19350]
        self.clean_text = ""


    def clean(self):
        print 'Start cleaning...'
        # Get rid of line breaks. 
        x = self.text.split("\n")
        x = [i.strip() for i in x]
        self.clean_text = " ".join(x)

        # Lower case.
        self.clean_text = self.clean_text.lower()

        # Remove punctuations
        tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                  if unicodedata.category(unichr(i)).startswith('P'))
        self.clean_text = self.clean_text.translate(tbl)

        # Remove spaces.
        self.clean_text = self.clean_text.replace("  "," ")
        while "  " in self.clean_text:
            self.clean_text = self.clean_text.replace("  "," ")
        print '   ...done'

    def lemmatize(self):
        print 'Start lemmatize...'
        # Lemmatize.
        wordnet_lemmatizer = WordNetLemmatizer()
        temp=""
        for word in self.clean_text.split():
            temp=temp+" "+wordnet_lemmatizer.lemmatize(word)

        self.clean_text = temp.strip() # get rid of white space. 
        print '   ...done'

    def stem(self):
        print "Start stem..."
        from nltk.stem.porter import PorterStemmer
        porter = PorterStemmer()
        temp=""
        for word in self.clean_text.split():
            temp=temp+" "+porter.stem(word)

        self.clean_text = temp.strip() # get rid of white space.
        print '   ...done'
        
    def __getitem__(self, index):
        if index == 0:
            return self.clean_text
        if index == 1:
            return self.title
        if index == 2:
            return self.author

   

In [12]:
# Load the corpus of books from a subdirectory. 
book_corpus = []
author = 'dumas'
for filename in os.listdir('./data/' + author):
    if filename[-3:] == 'txt':
        f = open('./data/' + author + '/' + filename)
        book = Book(f.readlines())
        f.close()
        book_corpus.append(book)

In [9]:
1238827 - 19350

1219477

In [10]:
len(book_corpus[0].text)

3344865

In [11]:
len(book_corpus[0].text)

3344865

In [12]:
print book_corpus[0].text[-19350:]

he page confessed that he was no farther
advanced than the day before; then the marquis scolded, threatened to
take away his fine clothes, to withdraw his own promises, and finally to
address himself to some other person. At this last threat the youth
would again call up his courage, and promise to be bolder to-morrow; and
on the morrow would spend the day in making a thousand compliments to
his mistress’s eyes, which she, in her innocence, did not understand. At
last, one day, Madame de Perrant asked him what made him look at her
thus, and he ventured to confess his love; but then Madame de Perrant,
changing her whole demeanour, assumed a face of sternness and bade him
go out of her room.

The poor lover obeyed, and ran, in despair, to confide his grief to the
husband, who appeared sincerely to share it, but consoled him by saying
that he had no doubt chosen his moment badly; that all women, even the
least severe, had inauspicious hours in which they would not yield to
attack, and tha

In [3]:
# Preprocess the text
for book in book_corpus:
    print book[1]
    book.clean()
    book.lemmatize()
    book.stem()
    print book[0][:100]

NameError: name 'book_corpus' is not defined

In [13]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')



In [14]:
# Define a function to split a book into parsed sentences
def book_to_sentences(book, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(book.text.encode("ascii","ignore").strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( book_to_wordlist( raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

def book_to_wordlist(book_text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    #  
    #  Decode from UTF-8
#     tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
#               if unicodedata.category(unichr(i)).startswith('P'))
#     book_text = book.text.translate(tbl)

    #
    # 3. Convert words to lower case and split themstring.decode('utf-8')
    words = book_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return words

In [15]:
sentences = []  # Initialize an empty list of sentences


print "Parsing sentences from training set"
for book in book_corpus:
    sentences += book_to_sentences(book, tokenizer)
    
    
# print "Parsing sentences from unlabeled set"
# for review in unlabeled_train["review"]:
#     sentences += review_to_sentences(review, tokenizer

Parsing sentences from training set


In [29]:
sentences[-10:]

[['but',
  'this',
  'monsieur',
  'of',
  'lintendance"',
  '(pointing',
  'over',
  'his',
  'shoulder',
  'to',
  'colbert,',
  'who',
  'if',
  'possible,',
  'became',
  'paler,',
  'behind',
  'him)',
  '"has',
  'in',
  'his',
  'coffers',
  'nine',
  'hundred',
  'thousand',
  'livres',
  'of',
  'mine."'],
 ['the', 'king', 'turned', 'round', 'to', 'look', 'at', 'colbert.'],
 ['"but--"', 'said', 'the', 'latter.'],
 ['"monsieur,"',
  'continued',
  'fouquet,',
  'still',
  'speaking',
  'indirectly',
  'to',
  'colbert,',
  '"monsieur',
  'has',
  'received,',
  'a',
  'week',
  'ago,',
  'sixteen',
  'hundred',
  'thousand',
  'livres;',
  'he',
  'has',
  'paid',
  'a',
  'hundred',
  'thousand',
  'livres',
  'to',
  'the',
  'guards,',
  'sixty-four',
  'thousand',
  'livres',
  'to',
  'the',
  'hospitals,',
  'twenty-five',
  'thousand',
  'to',
  'the',
  'swiss,',
  'an',
  'hundred',
  'and',
  'thirty',
  'thousand',
  'for',
  'provisions,',
  'a',
  'thousand',
  'fo

In [16]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = str(num_features)+'features_'+str(min_word_count)+'min_word_count_'+str(context)+'context.npy'
model.save(model_name)

2017-06-18 10:21:11,226 : INFO : collecting all words and their counts
2017-06-18 10:21:11,229 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-18 10:21:11,325 : INFO : PROGRESS: at sentence #10000, processed 324064 words, keeping 29516 word types
2017-06-18 10:21:11,388 : INFO : PROGRESS: at sentence #20000, processed 562931 words, keeping 41479 word types


Training model...


2017-06-18 10:21:11,444 : INFO : PROGRESS: at sentence #30000, processed 760320 words, keeping 49746 word types
2017-06-18 10:21:11,500 : INFO : PROGRESS: at sentence #40000, processed 928541 words, keeping 55250 word types
2017-06-18 10:21:11,552 : INFO : PROGRESS: at sentence #50000, processed 1090042 words, keeping 59820 word types
2017-06-18 10:21:11,621 : INFO : PROGRESS: at sentence #60000, processed 1246524 words, keeping 63973 word types
2017-06-18 10:21:11,680 : INFO : PROGRESS: at sentence #70000, processed 1395229 words, keeping 67527 word types
2017-06-18 10:21:11,727 : INFO : PROGRESS: at sentence #80000, processed 1558275 words, keeping 70709 word types
2017-06-18 10:21:11,779 : INFO : PROGRESS: at sentence #90000, processed 1706382 words, keeping 73504 word types
2017-06-18 10:21:11,823 : INFO : PROGRESS: at sentence #100000, processed 1846406 words, keeping 76168 word types
2017-06-18 10:21:11,868 : INFO : PROGRESS: at sentence #110000, processed 1981586 words, keeping 

In [8]:
model.doesnt_match("king queen sword".split())

'sword'

In [50]:
test_string = 'sword'
if test_string in model.wv.index2word:
    print 'True'
else:
    print 'False'

True


In [44]:
model.most_similar("king")[0]

('duke', 0.7709444761276245)

In [10]:
model.wv['sword'][0]

0.031802841

In [31]:
model.wv['sword'][0]

0.031802841

In [19]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    print words[:10]
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(books, model, num_features):
    # Given a set of books (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    bookFeatureVecs = np.zeros((len(books),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for book in books:
        # Print a status message every other book
        if counter%1. == 0.:
            print "Book %d of %d" % (counter, len(books))
        # 
        # Call the function (defined above) that makes average feature vectors
        bookFeatureVecs[counter] = makeFeatureVec(book, model, num_features)
        #
        # Increment the counter
        counter = counter + 1
    return bookFeatureVecs

In [20]:
getAvgFeatureVecs(book_corpus, model, num_features)

Book 0 of 8
None


OverflowError: iter index too large

In [13]:
for book in book_corpus:
    print book.text[:10]

quite unfi
her lover 
of the kin

“They des
“How so?”

but percei
even bowin
catching a


In [14]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_books = []
for book in book_corpus:
    clean_train_books.append( book_to_wordlist( book.text, remove_stopwords=True ))
trainDataVecs = getAvgFeatureVecs( clean_train_books, model, num_features )

# print "Creating average feature vecs for test reviews"
# clean_test_books = []
# for book in book_corpus:
#     print book
#     clean_test_books.append( book_to_wordlist( book.text, remove_stopwords=True ))

# testDataVecs = getAvgFeatureVecs( clean_test_book, model, num_features )

[u'quite', u'unfitted,', u'turned', u'eyes', u'towards', u'france,', u'victorious', u'fetes', u'rejoicings', u'awaiting']
Book 0 of 8
Book 1 of 8
Book 2 of 8
Book 3 of 8
Book 4 of 8
Book 5 of 8
Book 6 of 8
Book 7 of 8


In [44]:
clean_train_books[0][:10]

[u'quite',
 u'unfitted,',
 u'turned',
 u'eyes',
 u'towards',
 u'france,',
 u'victorious',
 u'fetes',
 u'rejoicings',
 u'awaiting']

In [46]:
trainDataVecs[0]

array([ 0.00819087,  0.00242723, -0.00023242,  0.00605961, -0.0115401 ,
        0.0226951 ,  0.00960517, -0.01059535,  0.01748   ,  0.00315369,
       -0.00691469, -0.02866292, -0.01269055,  0.00380171, -0.01082311,
        0.01059669, -0.01380262, -0.01193288, -0.00955732, -0.04420404,
       -0.02917563,  0.01162744,  0.02712551,  0.01107198,  0.00495543,
        0.02217433, -0.02694823, -0.05252416, -0.0280789 , -0.03110444,
       -0.01409325,  0.04262873,  0.00611702,  0.00018216,  0.00775476,
        0.00683201,  0.01443463,  0.00042482, -0.02677386, -0.02441705,
        0.00968186, -0.0175599 , -0.01889522, -0.00632703, -0.015441  ,
       -0.01178054, -0.01588683, -0.00518356,  0.01394426,  0.00999474,
       -0.01085228,  0.01466012,  0.00159153,  0.00867865,  0.00217163,
       -0.00634632,  0.00178571, -0.02554685, -0.0084363 , -0.02391442,
       -0.000587  ,  0.00486024,  0.0259176 , -0.03461701,  0.02162339,
       -0.01839769, -0.00831208,  0.03377576,  0.05346158, -0.01

In [65]:
import random

def rewrite_my_text(my_text, model):
    wordlist = model.wv.index2word
    punctuation = '.?!",;:()'.split()
    for mark in punctuation:
        my_text = my_text.replace(mark, ' ' + mark)
    lst = my_text.split()
    lst_new = []
    for word in lst:
        if (len(word) > 3) and (word in wordlist):
            word = word.strip()
            lst_new.append(model.most_similar(word)[random.randint(0,3)][0])
        else:
            lst_new.append(word)
    ret = ' '.join(lst_new)
    for mark in punctuation:
        ret = ret.replace(' ' + mark, mark)
    return ret

In [69]:
# For fun
MyText = "Your time is limited, so don't waste it living someone else's life. Don't be trapped by dogma - which is living with the results of other people's thinking. Don't let the noise of others' opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition."
rewrite_my_text(MyText, model)


'Your time, is limited, so don\'t waste it even, anybody else\'s future Don\'t be trapped by dogma - broken is suffering showed the results of other. people\'s thinking. Don\'t let the crowd of others\' opinions drown out "your own inner low And character important, had the strength to rejoin yourself grief and intuition.'